In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
%matplotlib inline

is_TH_measurement = False

is_Tel_Phot = True
# folder=tb.latest_data('lt3_SPCORR')

if is_TH_measurement:
    folder=tb.latest_data('SPCorrs_PSB')
else:
    folder=tb.latest_data('SPCorrs_ZPL')
     
print 'folder :', folder

name = 'ssro'
a = ppq.purifyPQAnalysis(folder, hdf5_mode='r')#, pq_folder='bs_remote')

In [ ]:
reload(pq_plots)
if is_TH_measurement:
    pq_plots.plot_photon_hist(a.pqf,start =3000,  length = 800, hist_binsize = 1, save = False, log=True) #start =7480
#     pq_plots.plot_photon_hist(a.pqf,start =0+TH_Ch1_offset_ns,  length = 3100, hist_binsize = 1, save = False, log=True) #start =7480
elif is_Tel_Phot:
    pq_plots.plot_marker_filter_comparison(
    a.pqf,start = 3920e3,mrkr_chan = 1,#,2.41E6,#+0.5e6,  
    length = 100e3, 
    hist_binsize = 1e3, 
    save = False, 
    log=False)
else:
    pq_plots.plot_marker_filter_comparison(
    a.pqf,start = 3770e3,mrkr_chan = 1,#,2.41E6,#+0.5e6,  
    length = 600e3, 
    hist_binsize = 1e3, 
    save = False, 
    log=True)

In [ ]:
sp_lt = a.pqf['/PQ_special-1'].value # Special: marker channel
ch_lt = a.pqf['/PQ_channel-1'].value # Channel: photon channel
sn_lt = a.pqf['/PQ_sync_number-1'].value # Sync number: the number of the last sync signal
st_lt = a.pqf['/PQ_sync_time-1'].value # Sync time: time that has passed since the last sync signal
#tt_lt = a.pqf['/PQ_time-1'].value 

In [ ]:
photon_channel = 1 # 0 or 1; 2 means both HH detectors   # psb doen't work if the PLU was active

if is_TH_measurement:
    st_start = 3070
    st_len       = 30 #50 ns
    st_len_w2    = 30
    p_sep        = 200 #XXX why has this changed?
elif is_Tel_Phot:
    st_start = 3941e3
    st_len       = 10e3 #50 ns
    st_len_w2    = 10e3
    p_sep        = 2*190e3 #XXX why has this changed?
else:
    st_start = 3780e3
    st_len       = 20e3 #50 ns
    st_len_w2    = 20e3
    p_sep        = 200e3 #XXX why has this changed?

In [ ]:
### filter the PQ data: Return an array which is True at each position where an event was in a window
if (photon_channel == 0) or (photon_channel == 1):
    st_fltr_w1 = (ch_lt == photon_channel)  & (st_lt > st_start)  & (st_lt < (st_start  + st_len)) & (sp_lt ==0)
    st_fltr_w2 = (ch_lt == photon_channel)  & (st_lt > st_start + p_sep)  & (st_lt < (st_start + p_sep + st_len_w2)) & (sp_lt ==0)
else:
    st_fltr_w1 = (st_lt > st_start)  & (st_lt < (st_start  + st_len)) & (sp_lt == 0)
    st_fltr_w2 = (st_lt > st_start + p_sep)  & (st_lt < (st_start + p_sep + st_len_w2)) & (sp_lt == 0)
#fltr_ro    = (sp_lt == 0) & (ch_lt == ro_channel)  & (st_lt > ro_start)  & (st_lt < (ro_start  + ro_length))   
print len(sn_lt[st_fltr_w1])
print len(sn_lt[st_fltr_w2])

In [ ]:
### filter the adwin RO data according to the now filtered pq data
noof_reps = len(np.unique(sn_lt))

adwin_filter_w1 = a.filter_adwin_data_from_pq_syncs(sn_lt[st_fltr_w1])
adwin_filter_w2 = a.filter_adwin_data_from_pq_syncs(sn_lt[st_fltr_w2])
noof_reps2= a.agrp['completed_reps'].value*a.g.attrs['LDE1_attempts']
noof_w1_ro0 = np.sum(a.agrp['ssro_results'].value[adwin_filter_w1[0]])
noof_w2_ro0 = np.sum(a.agrp['ssro_results'].value[adwin_filter_w2[0]])
#print len(a.filter_adwin_data_from_pq_syncs(st_fltr_w1)[1]), len(a.filter_adwin_data_from_pq_syncs(st_fltr_w2)[1])
    

w1_sn = sn_lt[st_fltr_w1]
w2_sn = sn_lt[st_fltr_w2]
noof_w1 = np.sum(st_fltr_w1)
noof_w2 = np.sum(st_fltr_w2)

print noof_w1,noof_w2

In [ ]:
print noof_reps,noof_reps2, noof_w1_ro0, noof_w2_ro0, noof_w1, noof_w2
print 'missed syncs', float(noof_reps)/noof_reps2
print 'tail_cts', float(noof_w1+noof_w2)/noof_reps2*1e4
print "RO's", noof_w1_ro0,noof_w2_ro0

In [ ]:
reload(ppq)
### load ssro!
ssro_calib_folder  = tb.latest_data('SSROCalib')
ro_time = a.agrp.attrs['E_RO_durations'][0]
print a.agrp.attrs['E_RO_durations'][0]
print ssro_calib_folder
trans = None # 'msm1'# only refers to ms1 states! see the function get_analysed_ssro_calibration. ('msm1' for ssro_mwinit.py) 
ssro_fids = ppq.get_analysed_ssro_calibration(ssro_calib_folder,e_transition = trans, readout_time = ro_time)
print ssro_fids

In [ ]:
# P_ms1 generally stands for the probability to measure ms=1
# P_photon is the probability for getting a photon
win1_P_photon = float(noof_w1_ro0)/noof_w1
u_win1_P_photon = np.sqrt(win1_P_photon * (1.-win1_P_photon)/noof_w1)

win2_P_photon = float(noof_w2_ro0)/noof_w2
u_win2_P_photon = np.sqrt(win2_P_photon * (1.-win2_P_photon)/noof_w2)
print noof_w1_ro0, noof_w1
print win1_P_photon,win2_P_photon, u_win1_P_photon, u_win2_P_photon
fig, (ax, ax_corr) = plt.subplots(1,2, figsize=(11,4),
                              sharex=True, sharey=True)

### uncorrected for e-SSRO
ax.bar(range(2), [win1_P_photon, win2_P_photon], color=['r', 'r'],
       align='center', yerr=[u_win1_P_photon, u_win2_P_photon], ecolor='k',
       width=0.8)

ax.set_xlim(-0.5,1.5)
ax.set_ylim(-0.05, 1.15)

ax.set_ylabel('P ($m_s =\, 0$)')

ax.set_xticks([0,1])
ax.set_xticklabels(['Photon in 1st window \n ({})'.format(noof_w1),
                    'Photon in 2nd window \n ({})'.format(noof_w2)])
ax.axvline(0.5, c='k', ls='--')
ax.axhline(0, c='k', ls=':')
ax.axhline(1, c='k', ls=':')

ax.text(0, win1_P_photon + 0.05, "{:.1f} $\pm$ {:.1f} %".format(win1_P_photon*100, u_win1_P_photon*100),
        va='bottom', ha='center',
        bbox = {'facecolor' : 'white', 'alpha' : 0.5})
ax.text(1, win2_P_photon + 0.05, "{:.1f} $\pm$ {:.1f} %".format(win2_P_photon*100, u_win2_P_photon*100),
        va='bottom', ha='center',
        bbox = {'facecolor' : 'white', 'alpha' : 0.5})
ax.set_title('Uncorrected readout')

### corrected for e-SSRO
win1_P0, u_win1_P0 = roc.single_qubit_outcome_with_ROC_from_fraction(win1_P_photon, u_win1_P_photon, 
                                                                     *ssro_fids)
win2_P0, u_win2_P0 = roc.single_qubit_outcome_with_ROC_from_fraction(win2_P_photon, u_win2_P_photon,
                                                                     *ssro_fids)

ax_corr.bar(range(2), [win1_P0, win2_P0], color=['b','b'],
       align='center', yerr=[u_win1_P0, u_win2_P0], ecolor='k',
       width=0.8)
ax_corr.axvline(0.5, c='k', ls='--')
ax_corr.axhline(0, c='k', ls=':')
ax_corr.axhline(1, c='k', ls=':')

ax_corr.text(0, win1_P0 + 0.05, "{:.1f} $\pm$ {:.1f} %".format(win1_P0*100, u_win1_P0*100),
        va='bottom', ha='center',
        bbox = {'facecolor' : 'white', 'alpha' : 0.5})
ax_corr.text(1, win2_P0 + 0.05, "{:.1f} $\pm$ {:.1f} %".format(win2_P0*100, u_win2_P0*100),
        va='bottom', ha='center',
        bbox = {'facecolor' : 'white', 'alpha' : 0.5})
ax_corr.set_title('Corrected readout \n '+a.timestamp)
fig.savefig(os.path.join(folder, 'Spin-photon_correlations.png'))

In [ ]:
hist(st_lt[fltr_ro][np.in1d(ro0_sn,w2_sn)])

In [ ]:
print 'PSB SP corr from date : {}'.format(tb.get_plot_title_from_folder(folder)[:15])
print '\tTail counts : {:.1f} '. format(float(noof_w1+noof_w2)/noof_reps2*1e4)
print '\tRatio of photons detected in w1/w2 : {:.1f} %'.format(np.float(noof_w1)/noof_w2*100)
print '\tUncorrected readout : W1 : {:.1f} +/- {:.1f} % & W2 : {:.1f} +/- {:.1f} %'.format(win1_P_photon*100, u_win1_P_photon*100,win2_P_photon*100, u_win2_P_photon*100)
print '\tCorrected readout : W1 : {:.1f} +/- {:.1f} % & W2 : {:.1f} +/- {:.1f} %'.format(win1_P0*100, u_win2_P0*100,win2_P0*100, u_win2_P0*100)
if win1_P_photon*100 <2.5 and win2_P_photon*100>93.5 and np.float(noof_w1)/noof_w2*100 > 94 and np.float(noof_w1)/noof_w2*100<106:
    print 'all thresholds (93.5 and 2.5 and 94:106) are met.'
else:
    print 'threasholds are NOT met. Compare to threshold: 93.5 and 2.5 and 94:106'